In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

In [2]:
target = pd.read_csv("../data/target_train.csv")
sample = pd.read_csv("../data/sample_submission.csv")

plavki_train = pd.read_csv("../data/plavki_train.csv")
plavki_test = pd.read_csv("../data/plavki_test.csv")

gas_train = pd.read_csv("../data/gas_train.csv")
gas_test = pd.read_csv("../data/gas_test.csv")

chugun_train = pd.read_csv("../data/chugun_train.csv")
chugun_test = pd.read_csv("../data/chugun_test.csv")

lom_train = pd.read_csv("../data/lom_train.csv")
lom_test = pd.read_csv("../data/lom_test.csv")

produv_train = pd.read_csv("../data/produv_train.csv")
produv_test = pd.read_csv("../data/produv_test.csv")

chronom_train = pd.read_csv("../data/chronom_train.csv")
chronom_test = pd.read_csv("../data/chronom_test.csv")

sip_train = pd.read_csv("../data/sip_train.csv")
sip_test = pd.read_csv("../data/sip_test.csv")

from pipeline import merge_data

params = {
    "chugun": {},
    "plavki": {"bow_count": 10},
    "vector_size": 10
}

train, test, y, num_features, cat_features = merge_data(
    sample, target, plavki_train, plavki_test, gas_train, gas_test, chugun_train, chugun_test, 
    lom_train, lom_test, produv_train, produv_test, chronom_train, chronom_test, 
    sip_train, sip_test, params
)

/home/data_sapiens/programs/projects/evraz/plavki/../lomproduv/features_edited.py:76: RuntimeWarning: divide by zero encountered in true_divide
  lom_train_transformed['ves_loma/ves_chuguna'] = lom_train_transformed['ves_loma'].values/chugun_train['VES'].values
/home/data_sapiens/programs/projects/evraz/plavki/../lomproduv/features_edited.py:82: RuntimeWarning: divide by zero encountered in true_divide
  lom_test_transformed['ves_loma/ves_chuguna'] = lom_test_transformed['ves_loma'].values/chugun_test['VES'].values


  0%|          | 0/2063 [00:00<?, ?it/s]

  0%|          | 0/780 [00:00<?, ?it/s]

In [104]:
from sklearn.model_selection import KFold, TimeSeriesSplit
from tqdm.auto import tqdm, trange

def metric(c_true, tst_true, c_pred, tst_pred, pwc=None, pwt=None):
    
    if pwc is not None:
        c_pred = pwc.inverse_transform(c_pred.reshape(-1, 1)).reshape(-1)
    if pwt is not None:
        tst_pred = pwt.inverse_transform(tst_pred.reshape(-1, 1)).reshape(-1)
    
    delta_c = np.abs(c_true - c_pred)
    hit_rate_c = np.int64(delta_c < 0.02)

    delta_t = np.abs(tst_true - tst_pred)
    hit_rate_t = np.int64(delta_t < 20)

    N = c_pred.shape[0]    
    return np.sum(hit_rate_c) / N, np.sum(hit_rate_t) / N, np.sum(hit_rate_c + hit_rate_t) / 2 / N

def pipeline(model_c, model_tst, train_c, test_c, train_tst, test_tst, y, sample, n_splits=10, pwc=None, pwt=None):
    kf = KFold(n_splits=n_splits, shuffle=True)

    sample["C"] = 0
    sample["TST"] = 0
    
    res_c, res_t, res = [], [], []
    for train_idx, test_idx in tqdm(kf.split(train_c), total=n_splits):
        cur_train_c = train_c[train_idx]
        cur_eval_c = train_c[test_idx]        

        cur_train_tst = train_tst[train_idx]
        cur_eval_tst = train_tst[test_idx]

        cur_train_y = y.iloc[train_idx]
        cur_eval_y = y.iloc[test_idx]
        
        if pwc is not None:
            if type(model_c) == CatBoostRegressor:
                model_c.fit(cur_train_c, pwc.transform(cur_train_y["C"].values.reshape(-1, 1)).reshape(-1),
                           eval_set=(cur_eval_c, pwc.transform(cur_eval_y["C"].values.reshape(-1, 1)).reshape(-1)),
                           early_stopping_rounds=35)
            else:
                model_c.fit(cur_train_c, pwc.transform(cur_train_y["C"].values.reshape(-1, 1)).reshape(-1))

        else:
            model_c.fit(cur_train_c, cur_train_y["C"])
            
        if pwt is not None:
            if type(model_tst) == CatBoostRegressor:
                model_tst.fit(cur_train_tst, pwt.transform(cur_train_y["TST"].values.reshape(-1, 1)).reshape(-1),
                              eval_set=(cur_eval_tst, pwc.transform(cur_eval_y["TST"].values.reshape(-1, 1)).reshape(-1)),
                              early_stopping_rounds=35)
            else:
                model_tst.fit(cur_train_tst, pwt.transform(cur_train_y["TST"].values.reshape(-1, 1)).reshape(-1))
        else:
            model_tst.fit(cur_train_tst, cur_train_y["TST"])
        
        eval_pred_c = model_c.predict(cur_eval_c)
        eval_pred_tst = model_tst.predict(cur_eval_tst)
    
        sample["C"] += model_c.predict(test_c) / n_splits
        sample["TST"] += model_tst.predict(test_tst) / n_splits
        
        hit_rate_c, hit_rate_t, hit_rate = metric(cur_eval_y["C"], cur_eval_y["TST"], eval_pred_c, eval_pred_tst, pwc, pwt)
        res_c.append(hit_rate_c)
        res_t.append(hit_rate_t)
        res.append(hit_rate)
    
    res_c = np.array(res_c)
    res_t = np.array(res_t)
    res = np.array(res)
    print(f"Carbon score: {res_c.mean()} ± {res_c.std()}")
    print(f"Temperature score: {res_t.mean()} ± {res_t.std()}")    
    print(f"Overall score: {res.mean()} ± {res.std()}") 
    
    if pwt is not None:
        sample["TST"] = pwt.inverse_transform(sample["TST"].values.reshape(-1, 1)).reshape(-1)
    if pwc is not None:
        sample["C"] = pwc.inverse_transform(sample["C"].values.reshape(-1, 1)).reshape(-1)
    
    sample["C"] = sample["C"].clip(0, 1)
    
    return sample, (res_c.mean(), res_t.mean(), res.mean())

In [17]:
from sklearn.preprocessing import PowerTransformer

pwrC = PowerTransformer('box-cox')
pwrT = PowerTransformer('box-cox')

pwrC2 = PowerTransformer()
pwrT2 = PowerTransformer()

y_exp = y.copy()

y_exp["C"] = pwrC.fit_transform(y['C'].values.reshape(-1, 1))
y_exp["TST"] = pwrT.fit_transform(y['TST'].values.reshape(-1, 1))

pwrC2.fit(y['C'].values.reshape(-1, 1))
pwrT2.fit(y['TST'].values.reshape(-1, 1))

PowerTransformer()

In [42]:
train_backup = train.copy()
test_backup = test.copy()

In [48]:
train = train_backup.copy()
test = test_backup.copy()

In [8]:
gas_filtered_for_C = pd.read_csv("../gas/gas_filtered_for_C.csv").set_index("NPLV")
gas_filtered_for_TST = pd.read_csv("../gas/gas_filtered_for_TST.csv").set_index("NPLV")

gas_test_autofeatures = pd.read_csv("../gas/gas_test_autofeatures.csv").set_index("NPLV")

In [93]:
train_gas_C = gas_filtered_for_C
train_gas_TST = gas_filtered_for_TST

test_gas_C = gas_test_autofeatures[gas_filtered_for_C.columns]
test_gas_TST = gas_test_autofeatures[gas_filtered_for_TST.columns]

In [94]:
train_C = train.join(train_gas_C, on="NPLV")
train_TST = train.join(train_gas_TST, on="NPLV")

test_C = test.join(test_gas_C, on="NPLV")
test_TST = test.join(test_gas_TST, on="NPLV")

In [54]:
from catboost import CatBoostRegressor

In [105]:
from lightgbm

In [ ]:
models = [
    CatBoostRegressor(n_estimators=500, grow_policy="Lossguide"),
    XGBRegressor(n_estimators=50, n_jobs=-1)
]

In [102]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

model_c = CatBoostRegressor(n_estimators=500, grow_policy="Lossguide")
model_tst = CatBoostRegressor(n_estimators=500, grow_policy="Lossguide")

# model_c = XGBRegressor(n_estimators=50, n_jobs=-1)
# model_tst = XGBRegressor(n_estimators=50, n_jobs=-1)

res = pipeline(model_c, model_tst, train_C[top_10_c].values, test_C[top_10_c].values, train_TST[top_10_tst].values, test_TST[top_10_tst].values, y, sample, 
               n_splits=200, pwc=pwrC, pwt=pwrT)

  0%|          | 0/200 [00:00<?, ?it/s]

Learning rate set to 0.081274
0:	learn: 0.9743220	test: 0.8282852	best: 0.8282852 (0)	total: 3.61ms	remaining: 1.8s
1:	learn: 0.9489149	test: 0.8074503	best: 0.8074503 (1)	total: 6.92ms	remaining: 1.72s
2:	learn: 0.9237050	test: 0.7825119	best: 0.7825119 (2)	total: 13.1ms	remaining: 2.17s
3:	learn: 0.8998448	test: 0.7600723	best: 0.7600723 (3)	total: 17ms	remaining: 2.11s
4:	learn: 0.8782732	test: 0.7389097	best: 0.7389097 (4)	total: 20.5ms	remaining: 2.03s
5:	learn: 0.8586844	test: 0.7080914	best: 0.7080914 (5)	total: 23.6ms	remaining: 1.95s
6:	learn: 0.8404247	test: 0.7006979	best: 0.7006979 (6)	total: 32.3ms	remaining: 2.27s
7:	learn: 0.8229554	test: 0.6863707	best: 0.6863707 (7)	total: 36.4ms	remaining: 2.24s
8:	learn: 0.8092531	test: 0.6725625	best: 0.6725625 (8)	total: 39.4ms	remaining: 2.15s
9:	learn: 0.7936992	test: 0.6522438	best: 0.6522438 (9)	total: 42.7ms	remaining: 2.09s
10:	learn: 0.7799884	test: 0.6351492	best: 0.6351492 (10)	total: 46.1ms	remaining: 2.05s
11:	learn: 0.7

52:	learn: 0.6441167	test: 2.6206629	best: 2.6124497 (32)	total: 135ms	remaining: 1.14s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 2.612449687
bestIteration = 32

Shrink model to first 33 iterations.
Learning rate set to 0.081274
0:	learn: 0.9714498	test: 0.9687872	best: 0.9687872 (0)	total: 3.55ms	remaining: 1.77s
1:	learn: 0.9469373	test: 0.9277123	best: 0.9277123 (1)	total: 7.19ms	remaining: 1.79s
2:	learn: 0.9247062	test: 0.8891559	best: 0.8891559 (2)	total: 16.3ms	remaining: 2.71s
3:	learn: 0.9014195	test: 0.8601333	best: 0.8601333 (3)	total: 20ms	remaining: 2.48s
4:	learn: 0.8783368	test: 0.8312111	best: 0.8312111 (4)	total: 23ms	remaining: 2.27s
5:	learn: 0.8602704	test: 0.8028530	best: 0.8028530 (5)	total: 26.4ms	remaining: 2.18s
6:	learn: 0.8413479	test: 0.7866356	best: 0.7866356 (6)	total: 29.6ms	remaining: 2.08s
7:	learn: 0.8243993	test: 0.7673038	best: 0.7673038 (7)	total: 32.5ms	remaining: 2s
8:	learn: 0.8088383	test: 0.7419004	best: 0.7419004 (8)	to

31:	learn: 0.6886860	test: 2.7241613	best: 2.7214504 (29)	total: 84.9ms	remaining: 1.24s
32:	learn: 0.6860613	test: 2.7244679	best: 2.7214504 (29)	total: 88.1ms	remaining: 1.25s
33:	learn: 0.6837228	test: 2.7193702	best: 2.7193702 (33)	total: 91ms	remaining: 1.25s
34:	learn: 0.6817848	test: 2.7172038	best: 2.7172038 (34)	total: 93.9ms	remaining: 1.25s
35:	learn: 0.6789432	test: 2.7167027	best: 2.7167027 (35)	total: 97.2ms	remaining: 1.25s
36:	learn: 0.6759064	test: 2.7213506	best: 2.7167027 (35)	total: 101ms	remaining: 1.26s
37:	learn: 0.6742758	test: 2.7170270	best: 2.7167027 (35)	total: 104ms	remaining: 1.27s
38:	learn: 0.6717481	test: 2.7103196	best: 2.7103196 (38)	total: 108ms	remaining: 1.27s
39:	learn: 0.6696388	test: 2.7133387	best: 2.7103196 (38)	total: 111ms	remaining: 1.27s
40:	learn: 0.6668872	test: 2.7191401	best: 2.7103196 (38)	total: 114ms	remaining: 1.27s
41:	learn: 0.6651065	test: 2.7199014	best: 2.7103196 (38)	total: 117ms	remaining: 1.27s
42:	learn: 0.6634753	test: 2.

Learning rate set to 0.081274
0:	learn: 0.9729957	test: 1.0729199	best: 1.0729199 (0)	total: 3.69ms	remaining: 1.84s
1:	learn: 0.9471664	test: 1.0378914	best: 1.0378914 (1)	total: 7.24ms	remaining: 1.8s
2:	learn: 0.9275547	test: 1.0173756	best: 1.0173756 (2)	total: 10.8ms	remaining: 1.79s
3:	learn: 0.9023098	test: 0.9877676	best: 0.9877676 (3)	total: 15.2ms	remaining: 1.88s
4:	learn: 0.8789686	test: 0.9711609	best: 0.9711609 (4)	total: 18.9ms	remaining: 1.87s
5:	learn: 0.8600300	test: 0.9457123	best: 0.9457123 (5)	total: 22.7ms	remaining: 1.87s
6:	learn: 0.8410780	test: 0.9274315	best: 0.9274315 (6)	total: 26.3ms	remaining: 1.85s
7:	learn: 0.8241933	test: 0.9167250	best: 0.9167250 (7)	total: 29.9ms	remaining: 1.84s
8:	learn: 0.8088599	test: 0.8932545	best: 0.8932545 (8)	total: 33.1ms	remaining: 1.8s
9:	learn: 0.7946558	test: 0.8662358	best: 0.8662358 (9)	total: 36.8ms	remaining: 1.8s
10:	learn: 0.7815139	test: 0.8490713	best: 0.8490713 (10)	total: 40.3ms	remaining: 1.79s
11:	learn: 0.7

...

103:	learn: 0.5139325	test: 0.6803474	best: 0.6794915 (95)	total: 404ms	remaining: 1.54s
104:	learn: 0.5130765	test: 0.6748637	best: 0.6748637 (104)	total: 407ms	remaining: 1.53s
105:	learn: 0.5118627	test: 0.6746268	best: 0.6746268 (105)	total: 410ms	remaining: 1.52s
106:	learn: 0.5111861	test: 0.6739051	best: 0.6739051 (106)	total: 413ms	remaining: 1.52s
107:	learn: 0.5097195	test: 0.6748314	best: 0.6739051 (106)	total: 416ms	remaining: 1.51s
108:	learn: 0.5081966	test: 0.6742446	best: 0.6739051 (106)	total: 420ms	remaining: 1.5s
109:	learn: 0.5073728	test: 0.6742793	best: 0.6739051 (106)	total: 423ms	remaining: 1.5s
110:	learn: 0.5058347	test: 0.6744751	best: 0.6739051 (106)	total: 427ms	remaining: 1.5s
111:	learn: 0.5032225	test: 0.6752157	best: 0.6739051 (106)	total: 430ms	remaining: 1.49s
112:	learn: 0.5030183	test: 0.6751047	best: 0.6739051 (106)	total: 434ms	remaining: 1.48s
113:	learn: 0.5013714	test: 0.6720849	best: 0.6720849 (113)	total: 437ms	remaining: 1.48s
114:	learn: 0.

39:	learn: 0.6131704	test: 0.7993145	best: 0.7993145 (39)	total: 136ms	remaining: 1.56s
40:	learn: 0.6112858	test: 0.7996861	best: 0.7993145 (39)	total: 140ms	remaining: 1.57s
41:	learn: 0.6092118	test: 0.7995387	best: 0.7993145 (39)	total: 143ms	remaining: 1.56s
42:	learn: 0.6070756	test: 0.7954033	best: 0.7954033 (42)	total: 147ms	remaining: 1.56s
43:	learn: 0.6051329	test: 0.7976585	best: 0.7954033 (42)	total: 151ms	remaining: 1.56s
44:	learn: 0.6030128	test: 0.7956418	best: 0.7954033 (42)	total: 156ms	remaining: 1.58s
45:	learn: 0.6011172	test: 0.7967496	best: 0.7954033 (42)	total: 159ms	remaining: 1.57s
46:	learn: 0.5987689	test: 0.7962485	best: 0.7954033 (42)	total: 163ms	remaining: 1.57s
47:	learn: 0.5962935	test: 0.7969482	best: 0.7954033 (42)	total: 167ms	remaining: 1.57s
48:	learn: 0.5939210	test: 0.7886498	best: 0.7886498 (48)	total: 171ms	remaining: 1.57s
49:	learn: 0.5919749	test: 0.7897746	best: 0.7886498 (48)	total: 175ms	remaining: 1.57s
50:	learn: 0.5892999	test: 0.786

85:	learn: 0.6004534	test: 2.6211071	best: 2.6097350 (75)	total: 253ms	remaining: 1.22s
86:	learn: 0.5992164	test: 2.6225010	best: 2.6097350 (75)	total: 255ms	remaining: 1.21s
87:	learn: 0.5980155	test: 2.6215936	best: 2.6097350 (75)	total: 258ms	remaining: 1.21s
88:	learn: 0.5969029	test: 2.6214473	best: 2.6097350 (75)	total: 261ms	remaining: 1.21s
89:	learn: 0.5957829	test: 2.6209499	best: 2.6097350 (75)	total: 265ms	remaining: 1.2s
90:	learn: 0.5949985	test: 2.6207827	best: 2.6097350 (75)	total: 268ms	remaining: 1.2s
91:	learn: 0.5942634	test: 2.6195253	best: 2.6097350 (75)	total: 271ms	remaining: 1.2s
92:	learn: 0.5926232	test: 2.6208879	best: 2.6097350 (75)	total: 275ms	remaining: 1.2s
93:	learn: 0.5913909	test: 2.6191813	best: 2.6097350 (75)	total: 278ms	remaining: 1.2s
94:	learn: 0.5901623	test: 2.6168486	best: 2.6097350 (75)	total: 282ms	remaining: 1.2s
95:	learn: 0.5886247	test: 2.6166292	best: 2.6097350 (75)	total: 285ms	remaining: 1.2s
Stopped by overfitting detector  (20 it

9:	learn: 0.7995803	test: 2.7529290	best: 2.7529290 (9)	total: 26.2ms	remaining: 1.28s
10:	learn: 0.7896650	test: 2.7454874	best: 2.7454874 (10)	total: 29.1ms	remaining: 1.29s
11:	learn: 0.7782981	test: 2.7464328	best: 2.7454874 (10)	total: 32.1ms	remaining: 1.3s
12:	learn: 0.7676040	test: 2.7508747	best: 2.7454874 (10)	total: 35ms	remaining: 1.31s
13:	learn: 0.7603115	test: 2.7515923	best: 2.7454874 (10)	total: 38.1ms	remaining: 1.32s
14:	learn: 0.7543030	test: 2.7530530	best: 2.7454874 (10)	total: 41ms	remaining: 1.33s
15:	learn: 0.7487984	test: 2.7549397	best: 2.7454874 (10)	total: 43.7ms	remaining: 1.32s
16:	learn: 0.7418646	test: 2.7580487	best: 2.7454874 (10)	total: 46.5ms	remaining: 1.32s
17:	learn: 0.7361977	test: 2.7562807	best: 2.7454874 (10)	total: 49.5ms	remaining: 1.32s
18:	learn: 0.7313766	test: 2.7559637	best: 2.7454874 (10)	total: 52.3ms	remaining: 1.32s
19:	learn: 0.7268348	test: 2.7569861	best: 2.7454874 (10)	total: 56.7ms	remaining: 1.36s
20:	learn: 0.7227360	test: 2

19:	learn: 0.7313988	test: 2.8149693	best: 2.7782553 (5)	total: 51.6ms	remaining: 1.24s
20:	learn: 0.7273035	test: 2.8147488	best: 2.7782553 (5)	total: 54.4ms	remaining: 1.24s
21:	learn: 0.7231521	test: 2.8127521	best: 2.7782553 (5)	total: 57.8ms	remaining: 1.26s
22:	learn: 0.7185218	test: 2.8125829	best: 2.7782553 (5)	total: 60.8ms	remaining: 1.26s
23:	learn: 0.7132263	test: 2.8144414	best: 2.7782553 (5)	total: 63.8ms	remaining: 1.26s
24:	learn: 0.7096104	test: 2.8122981	best: 2.7782553 (5)	total: 66.7ms	remaining: 1.27s
25:	learn: 0.7049419	test: 2.8127977	best: 2.7782553 (5)	total: 69.9ms	remaining: 1.27s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 2.778255278
bestIteration = 5

Shrink model to first 6 iterations.
Carbon score: 0.7506363636363637 ± 0.12941863010471802
Temperature score: 0.5830454545454545 ± 0.17565856602563343
Overall score: 0.6668409090909091 ± 0.11904735280863578


In [98]:
depthwise_res = res.copy()

In [89]:
xgb_res = res.copy()

In [73]:
res_catboost = res.copy()

In [75]:
res["C"] = res_catboost["C"]

In [91]:
xgb_res["C"] = res["C"]

In [95]:
model_c = XGBRegressor(n_estimators=100, n_jobs=-1)
model_c.fit(train_C, y["C"])

model_tst = XGBRegressor(n_estimators=100, n_jobs=-1)
model_tst.fit(train_TST, y["TST"])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [53]:
train_C.to_csv("train_C.csv", index=False)
train_TST.to_csv("train_TST.csv", index=False)

y.to_csv("y.csv", index=False)

In [79]:
top_10_c = [el[1] for el in sorted(zip(model_c.feature_importances_, train_C.columns), reverse=True)[:10]]
top_10_tst = [el[1] for el in sorted(zip(model_tst.feature_importances_, train_TST.columns), reverse=True)[:10]]

In [100]:
top_10_c = [el[1] for el in sorted(zip(model_c.feature_importances_, train_C.columns), reverse=True)[:10]]
top_10_c

['CO2__sum_values',
 'N2__quantile__q_0.9',
 'N2__quantile__q_0.7',
 'w2v_11',
 'O2__count_below_mean',
 'truncated_NMZ',
 'CO2__change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.0',
 'CO2__index_mass_quantile__q_0.9',
 'O2_опер_0',
 'AR__autocorrelation__lag_8']

In [86]:
for i, col in enumerate(top_10_c):
    for col2 in top_10_c[i:]:
        train_C[col + "_mult_" + col2] = train_C[col] * train_C[col2]
        test_C[col + "_mult_" + col2] = test_C[col] * test_C[col2]
        
for i, col in enumerate(top_10_tst):
    for col2 in top_10_tst[i:]:
        train_TST[col + "_mult_" + col2] = train_TST[col] * train_TST[col2]
        test_TST[col + "_mult_" + col2] = test_TST[col] * test_TST[col2]

In [88]:
sorted(zip(model_c.feature_importances_, train_C.columns), reverse=True)[:50]

[(0.14433177, 'w2v_11_mult_CO2__sum_values'),
 (0.06826114, 'truncated_NMZ'),
 (0.03270061, 'w2v_11_mult_CO2__index_mass_quantile__q_0.9'),
 (0.031435657,
  'CO2__index_mass_quantile__q_0.9_mult_T фурмы 2__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.4'),
 (0.02900526, 'O2__count_below_mean_mult_w2v_11'),
 (0.028511364, 'total_operations_опер_0'),
 (0.025931792, 'AR__autocorrelation__lag_7'),
 (0.025807366, 'max_duration_опер_0'),
 (0.023808384,
  'O2__count_below_mean_mult_T фурмы 2__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.4'),
 (0.019393198,
  'T__agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"max"'),
 (0.019196622, 'AR__autocorrelation__lag_8'),
 (0.017442172,
  'O2__count_below_mean_mult_CO2__change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.0'),
 (0.016893273, 'CO2__index_mass_quantile__q_0.9'),
 (0.013134967, 'CO2__index_mass_quantile__q_0.8'),
 (0.013083274, 'VES'),
 (0.0125474045, 'AR__quantile__q_0.9'),
 (0.011426015, 'gas_mean_CO2_mult

In [92]:
xgb_res.to_csv("joint2.csv", index=False)

In [103]:
test_TST.to_csv("test_TST.csv", index=False)
test_C.to_csv("test_C.csv", index=False)